#### **Penjelasan**

**Note** : *jika ada yang tidak dipahami, mohon jangan sungkan untuk menanyakan atau mengklarifikasikan dengan kami lagi ya kak*

Alur pengerjaan notebook ini adalah sebagai berikut:
- **Lakukan preprocessing pada data unlabeled** kemudian data hasil preprocessing disimpan untuk digunakan pada tahap pelabelan data
- **Lakukan training model machine learning** dengan menggunakan model pretrained IndoBERT. Pelatihan model dilakukan sebanyak 10 kali dengan menggunakan nilai parameter yang berbeda (batch size, learning rate, dan jumlah epoch). Untuk informasi mengenai parameter-parameter yang digunakan, kami memberikan informasinya seperti dibawah ini.

berikut dibawah ini jenis parameter yang digunakan :
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 64.

Setiap selesai pelatihan model, langsung gunakan model untuk memberikan label pada data unlabeled. Hasil labeling disimpan ke dalam file csv karena nantinya hasil csv tersebut akan digunakan untuk voting.

- **Load seluruh data voting**
- **Jalankan kode program yang berfungsi untuk melakukan voting**
- Dikarenakan mungkin nanti bakalan terdapat voting yang seimbang, artinya class yang memiliki frekuensi terbanyak terdapat lebih dari satu, maka data tersebut akan diberikan label oleh model yang memiliki tingkat akurasi tertinggi pada saat pelatihan model. Model tersebut adalah model dengan parameter epoch 3, learning rate 2e-5, dan batch size 64. 

#### **Preprocessing pada Data Unlabeled**

Pada tahap awal, kita akan melakukan **Preprocessing** terlebih dahulu pada **Data Unlabeled** agar dapat dipersiapkan untuk diberikan label

In [ ]:
import re
import csv
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read unlabeled data
df = pd.read_excel('dataset_unlabeled_penyisihan_bdc_2024.xlsx')
df.head()

,IDText,Text
0,TXT0001,"Lu mau org2 pro-demokrasi di negara ini bisa punya kesempatan lagi buat ngelawan oligarki di indo?\n\n- Pilih AMIN, tpi jangan pilih PKS, pilih Partai Buruh\n- Jangan pilih Prabowo dan semua partai pengusungnya\n- Pilih GAMA, tpi jangan pilih PDIP, pilih Partai Buruh [RE debordsbitch]"
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia menjawab hutang luar negeri Indonesia masih terhitung rendah dan dia tak khawatir intervensi asing akibat hutang seperti krisis moneter 1997 saat Orde Baru.\n\nDibantah Ganjar hutang luar negeri demi infrastruktur bisa berbahaya. \n(BTW bajunya Ganjar matching dengan tema debat)\n\nAnies menegaskan normal hutang luar negeri yg aman tak lebih dari 30% GDP. Hutang dipakai Menhan membeli alutsista bekas adalah suatu kesia-siaan.\n\nPrabowo tak merespon soal alutsista bekas karena jika membeli baru uang yg dikeluarkan lebih besar. [RE mazzini_gsp]
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau sosok yang mengagunhkan nilai nilai pancasila dan banyak memberikan kesadaran yg baik buat kita semua #GanjarMerawatPancasila\n\n #Mahfud Hebat #MahfudLebihBaik3 #GanjarMahfud2024 #3MandatGanjarMahfud #L3bihbaik Happy New Year
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan itu semua demi kesejahteraan rakyat
4,TXT0005,"@sniperruben45 @uda_zulhendra @ainunnajib Lah justru yg gak nyambung junjungan elu aomkmkmkmk\n\nNgomongin etika tapi ngatain Pak Anies omon omon sambil nyenyek tuh gimana? ÃƒÂ°Ã‚Å¸Ã‚Â¤Ã‚Â£ Pak Anies masih sopan lho, gak ngakak brutal atau pasang wajah mengejek waktu dia asbun hutang 50% atau nyuruh belajar ekonomi lagi."


In [ ]:
def remove_noise_from_text(text):
  text = re.sub(r'@[A-Za-z0-9_/=]+', ' [username] ', text)   # mengganti mention (misalnya, @username) dengan [username]
  #text = re.sub(r'#[A-Za-z0-9_]+', ' ', text)   # mengganti hashtag (misalnya #covid19) dengan spasi
  text = re.sub(r'https?://\S+', ' [link_url] ', text)     # mengganti URL dengan spasi
  return text
  #........

def preprocess_text(text):
    text = re.sub(r'^[a-zA-Z0-9_@?:/]\s', ' ', text)                 # menghapus tanda baca (sepertinya mengubahnya menjadi spasi saja)
    text = re.sub(r'\s+', ' ', text).strip()             # mengganti spasi berlebih dengan satu spasi
    return text

In [ ]:
df['Text'] = df['Text'].apply(preprocess_text)

In [ ]:
df

,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau sosok...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...
...,...,...
995,TXT0996,"Bikin bangga deh, Ganjar-Mahfud mau alokasikan..."
996,TXT0997,Pak Jokowi sebelum pilpres 2024 berbesar hati ...
997,TXT0998,@datuakrajoangek Sbaiknya si gemot nga usah ik...
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...


In [ ]:
df['Text'] = df['Text'].apply(remove_noise_from_text)

In [ ]:
df

,IDText,Text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau sosok...
3,TXT0004,[username] Prabowo Gibran yang bisa melakuka...
4,TXT0005,[username] [username] [username] Lah jus...
...,...,...
995,TXT0996,"Bikin bangga deh, Ganjar-Mahfud mau alokasikan..."
996,TXT0997,Pak Jokowi sebelum pilpres 2024 berbesar hati ...
997,TXT0998,[username] Sbaiknya si gemot nga usah ikutan...
998,TXT0999,kebiasaan merembuk atau bermusyawarah jadi gay...


In [ ]:
# Export unlabeled data to CSV
df.to_csv("data_unlabeled.csv", index=False)

Selanjutnya, kita akan mulai melatih model Machine Learningnya dan langsung melakukan prediksi label pada data unlabeled. **Perlu diingat**, hasil label disimpan ke csv terpisah karena akan digunakan pada saat melakukan voting.

In [ ]:
import re
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertModel, BertTokenizer,  AdamW, get_linear_schedule_with_warmup

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
from wordcloud import WordCloud

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pd.set_option('display.max_colwidth', None)

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
# Load data training 
df = pd.read_csv("dataset_penyisihan_bdc_2024.csv")
df.head()

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik. #IndonesiaSentris #IndonesiaHijau #02Melanjutkan #AnakMudaIndonesiaEmas Prabowo Subianto,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi Rektor mewajibkan mata kuliah anti-korupsi untuk memutus mata rantai korupsi.. #AminMiskinkanKoruptor [link_url] [RE ekowboy2],Politik
2,"[username] emng bener sih, pendukung 01 ada yg goblok, begitu jg dg pendukung 02.. hnya sj menurut pak Ridwan Kamil skemanya terbalik, klo 01 MAYORITAS PENDIDIKAN MENENGAH ATAS (artinya ada jg pendidikan rendah yg milih",Demografi
3,"RT [username] Sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua, giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras. Apakah ini tidak standar ganda? ÃƒÂ°Ã‚Å¸Ã‚Â¤Ã‚Â­ [RE coldthem]",Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polri Pegang Sumpahnya dalam Pemilu [link_url],Politik


In [ ]:
df['label'].value_counts()

label
Politik                    2972
Sosial Budaya               587
Ideologi                    400
Pertahanan dan Keamanan     400
Ekonomi                     367
Sumber Daya Alam            192
Demografi                    62
Geografi                     20
Name: count, dtype: int64

In [ ]:
df.shape

(5000, 2)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords = set(nltk.corpus.stopwords.words('indonesian'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove RT (retweet tags)
    text = re.sub(r'\brt\b', '', text)

    # Remove usernames enclosed in square brackets [username]
    text = re.sub(r'\[\w+\]', '', text)

    # Remove special characters and numbers (keep only letters, numbers, #, and punctuation)
    text = re.sub(r'[^a-zA-Z#@,\s]', '', text)

    # Tokenize using NLTK tokenizer
    tokens = word_tokenize(text)

    # Remove stopwords (if desired, uncomment the line below)
    # tokens = [token for token in tokens if token not in stopwords]

    # Join tokens back into a single string
    text = ' '.join(tokens)

    return text.strip()  # Remove leading/trailing spaces if any
# Apply preprocessing function to 'text' column in DataFrame
df['text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df

,text,label
0,kunjungan prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik # indonesiasentris # indonesiahijau # melanjutkan # anakmudaindonesiaemas prabowo subianto,Sumber Daya Alam
1,anies dapat tepuk tangan meriah saat jadi rektor mewajibkan mata kuliah antikorupsi untuk memutus mata rantai korupsi # aminmiskinkankoruptor re ekowboy,Politik
2,"emng bener sih , pendukung ada yg goblok , begitu jg dg pendukung hnya sj menurut pak ridwan kamil skemanya terbalik , klo mayoritas pendidikan menengah atas artinya ada jg pendidikan rendah yg milih",Demografi
3,"sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua , giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras apakah ini tidak standar ganda re coldthem",Politik
4,anies baswedan harap asn termasuk tni dan polri pegang sumpahnya dalam pemilu,Politik
...,...,...
4995,"ngeliat debat kemaren pas prabowo kicep , kekira keluarga yg biasa lakukan aksi kamisan di depan istana re binnuh",Politik
4996,masyarakat yakin bahwa prabowogibran memiliki visi yang sejalan dengan aspirasi rakyat # kosongduasatuputaran rakyatsenang menang,Politik
4997,imo both are irrational but yg satu jauh lebih irrational tbh but nevertheless dua duanya mesti diluruskan dan baik anies dan ahok kudu terlibat langsung untuk validasi trauma mereka dengan empati lho ya gitu ae sih,Ekonomi
4998,"look at that pak ganjar , anda sdh berkecimpung di lgislatif , eksekutif bhkan capres saja mamatgs brani mngmcam , ancaman itu nyata pak ganjar , nyalakan tanda bahaya utk opm operasi persenjataan mamat",Pertahanan dan Keamanan


In [ ]:
class_names = ['Sumber Daya Alam', 'Politik', 'Demografi', 'Pertahanan dan Keamanan',
               'Ideologi', 'Ekonomi', 'Sosial Budaya', 'Geografi']

df.label = df.label.map({'Sumber Daya Alam':0,
                         'Politik':1, 'Demografi':2,
                         'Pertahanan dan Keamanan':3,
                         'Ideologi':4, 'Ekonomi':5,
                         'Sosial Budaya':6, 'Geografi':7})

no_of_classes = 8

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        review = str(self.texts[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = CustomDataset(
        texts=df.text.to_numpy(),
        targets=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2
    )


In [ ]:
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-large-p2")
PRE_TRAINED_MODEL_NAME = "indobenchmark/indobert-large-p2"
# FINE_TUNED_MODEL_NAME = '/content/indobert_largep_2_ep2lr2e-5bs64.bin'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Tokenize texts to get token lengths
token_lengths = [len(tokenizer.encode(text)) for text in df['text']]

# Calculate max_len based on your criteria (e.g., 95th percentile)
max_len = int(np.percentile(token_lengths, 96))

print(f"Recommended max_len: {max_len}")

Recommended max_len: 99


Ganti **BATCH SIZE** dan **EPOCHS** dengan parameter berbeda

lakukan training model sebanyak 10 kali. berikut dibawah ini jenis parameter yang digunakan :
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 64.

In [ ]:
MAX_LEN = 99
BATCH_SIZE = 64   #batch size
EPOCHS = 2        #epoch

df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.2)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(pooled_output[1])
        return self.out(output)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(df_train['label']), y=df_train['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

Ganti **LEARNING RATE** dengan parameter yang berbeda

lakukan training model sebanyak 10 kali. berikut dibawah ini jenis parameter yang digunakan :
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 10, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 2e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 2, learning rate 3e-5, dan batch size 64.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 1e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 32.
- menggunakan model IndoBERT Large P2 dengan epoch 3, learning rate 2e-5, dan batch size 64.

In [ ]:
model = TextClassifier(no_of_classes).to(device)
                                    #learning rate
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.train()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_targets = []

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())

        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    balanced_acc = balanced_accuracy_score(all_targets, all_preds)
    return correct_predictions.double() / n_examples, np.mean(losses), balanced_acc

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.eval()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    balanced_acc = balanced_accuracy_score(all_targets, all_preds)
    return correct_predictions.double() / n_examples, np.mean(losses), balanced_acc

In [ ]:
history = defaultdict(list)
best_accuracy = 0
best_balanced_accuracy = 0

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss, train_balanced_acc = train_epoch(
        model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
    )
    print(f'Train loss {train_loss} accuracy {train_acc} balanced accuracy {train_balanced_acc}')

    val_acc, val_loss, val_balanced_acc = eval_model(
        model, val_data_loader, loss_fn, device, len(df_val)
    )
    print(f'Val loss {val_loss} accuracy {val_acc} balanced accuracy {val_balanced_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['train_balanced_acc'].append(train_balanced_acc)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    history['val_balanced_acc'].append(val_balanced_acc)

    # if val_balanced_acc > best_balanced_accuracy:
    #     best_balanced_accuracy = val_balanced_acc
    #     model_to_save = model.module if hasattr(model, 'module') else model
    #     torch.save(model_to_save.state_dict(), FINE_TUNED_MODEL_NAME + '_balanced_acc')

##### **Load Unlabeled Data**

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel

# Load data yang tidak ada label
unlabeled_df = pd.read_csv("data_test.csv")

In [ ]:
nltk.download('punkt')
stopwords = set(nltk.corpus.stopwords.words('indonesian'))

def preprocess_text(Text):
    # Convert to lowercase
    Text = Text.lower()
    # Remove URLs
    Text = re.sub(r'http\S+', '', Text)
    # Remove RT (retweet tags)
    Text = re.sub(r'\brt\b', '', Text)
    # Remove usernames enclosed in square brackets [username]
    Text = re.sub(r'\[\w+\]', '', Text)
    # Remove special characters and numbers (keep only letters, numbers, #, and punctuation)
    Text = re.sub(r'[^a-zA-Z#@,\s]', '', Text)
    # Tokenize using NLTK tokenizer
    Tokens = word_tokenize(Text)

    # Remove stopwords (if desired, uncomment the line below)
    # tokens = [token for token in tokens if token not in stopwords]

    # Join tokens back into a single string
    Text = ' '.join(Tokens)

    return Text.strip()  # Remove leading/trailing spaces if any

In [ ]:
# Apply preprocessing function to 'text' column in DataFrame
unlabeled_df['Text'] = unlabeled_df['Text'].apply(preprocess_text)

In [ ]:
# Define a function to preprocess the text
def preprocess_data(texts):
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    return encodings

# Preprocess the texts
texts = unlabeled_df['Text'].tolist()
encodings = preprocess_data(texts)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

with torch.no_grad():
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

predictions = torch.argmax(outputs, dim=1).cpu().numpy()



# Map predictions to class names
class_names = ['Sumber Daya Alam', 'Politik', 'Demografi', 'Pertahanan dan Keamanan',
                'Ideologi', 'Ekonomi', 'Sosial Budaya', 'Geografi']

predicted_labels = [class_names[pred] for pred in predictions]

# Add predicted labels to DataFrame
unlabeled_df['label'] = predicted_labels
labeled_df = unlabeled_df

# Save labeled DataFrame to CSV or perform further analysis   ## UBAH KATA-KATA DISINI
labeled_df.to_csv("hasil csv/predict using indobert large p2 ep2lr2e-5bs64.csv", index=False)

In [ ]:
labeled_df['label'].value_counts()

In [ ]:
train_acc = []
for e in history['train_acc']:
    train_acc.append(e.cpu().numpy())
val_acc = []
for e in history['val_acc']:
    val_acc.append(e.cpu().numpy())
plt.plot(train_acc, label='train accuracy')
plt.plot(val_acc, label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1]);

In [ ]:
import torch
import torch.nn.functional as F

def get_predictions(model, data_loader):
    model = model.eval()

    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            texts = d["review_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            probs = F.softmax(outputs, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds.cpu().tolist())
            prediction_probs.extend(probs.cpu().tolist())
            real_values.extend(targets.cpu().tolist())

    return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_data_loader
)

ypred = y_pred
ytest = df_test['label']

Setelah melatih beberapa model machine learning dengan parameter yang berbeda-beda dan memberikan label ke data unlabeled, gunakan informasi kolom hasil label dari kesemua jenis model dan jalankan skrip python yang berfungsi untuk melakukan voting pemberian label pada data unlabeled

In [ ]:
import os
import pandas as pd
from collections import Counter

In [ ]:
csv_files = [
    "hasil csv/predict using indobert large p2 ep3lr2e-5bs32.csv",   ######
    "hasil csv/predict using indobert large p2 ep10lr1e-5bs32.csv",
    "hasil csv/predict using indobert large p2 ep10lr2e-5bs32.csv",
    "hasil csv/predict using indobert large p2 ep2lr2e-5bs32.csv",
    "hasil csv/predict using indobert large p2 ep2lr2e-5bs64.csv",
    "hasil csv/predict using indobert large p2 ep2lr3e-5bs32.csv",
    "hasil csv/predict using indobert large p2 ep2lr3e-5bs64.csv",
    "hasil csv/predict using indobert large p2 ep3lr1e-5bs32.csv",
    "hasil csv/predict using indobert large p2 ep3lr2e-5bs32.csv",   ######
    "hasil csv/predict using indobert large p2 ep3lr2e-5bs64.csv",   ######
]

In [ ]:
def get_file_name(file_path):
    return os.path.splitext(os.path.basename(file_path))[0]

In [ ]:
predictions = [pd.read_csv(file)['label'] for file in csv_files]
column_names = [get_file_name(file) for file in csv_files]

In [ ]:
combined_predictions = pd.DataFrame(predictions).transpose()
combined_predictions.columns = column_names

In [ ]:
combined_predictions

,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep10lr1e-5bs32,predict using indobert large p2 ep10lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs64,predict using indobert large p2 ep2lr3e-5bs32,predict using indobert large p2 ep2lr3e-5bs64,predict using indobert large p2 ep3lr1e-5bs32,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep3lr2e-5bs64
0,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Ideologi
1,Politik,Politik,Politik,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Politik,Pertahanan dan Keamanan
2,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi
3,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik
4,Ekonomi,Ekonomi,Sosial Budaya,Sosial Budaya,Ekonomi,Sosial Budaya,Ekonomi,Ekonomi,Ekonomi,Ekonomi
...,...,...,...,...,...,...,...,...,...,...
995,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi
996,Politik,Politik,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,Politik,Ideologi
997,Politik,Politik,Politik,Politik,Politik,Politik,Pertahanan dan Keamanan,Politik,Politik,Politik
998,Ekonomi,Politik,Politik,Ekonomi,Ekonomi,Ekonomi,Politik,Ekonomi,Ekonomi,Ekonomi


In [ ]:
sample_code = pd.read_csv("hasil csv/predict using indobert large p2 ep3lr2e-5bs32.csv")['IDText']

In [ ]:
sample_code

0      TXT0001
1      TXT0002
2      TXT0003
3      TXT0004
4      TXT0005
        ...   
995    TXT0996
996    TXT0997
997    TXT0998
998    TXT0999
999    TXT1000
Name: IDText, Length: 1000, dtype: object

In [ ]:
combined_predictions['IDText'] = sample_code

In [ ]:
combined_predictions

,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep10lr1e-5bs32,predict using indobert large p2 ep10lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs64,predict using indobert large p2 ep2lr3e-5bs32,predict using indobert large p2 ep2lr3e-5bs64,predict using indobert large p2 ep3lr1e-5bs32,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep3lr2e-5bs64,IDText
0,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Ideologi,TXT0001
1,Politik,Politik,Politik,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Pertahanan dan Keamanan,Politik,Pertahanan dan Keamanan,TXT0002
2,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,TXT0003
3,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,TXT0004
4,Ekonomi,Ekonomi,Sosial Budaya,Sosial Budaya,Ekonomi,Sosial Budaya,Ekonomi,Ekonomi,Ekonomi,Ekonomi,TXT0005
...,...,...,...,...,...,...,...,...,...,...,...
995,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,TXT0996
996,Politik,Politik,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,Politik,Ideologi,TXT0997
997,Politik,Politik,Politik,Politik,Politik,Politik,Pertahanan dan Keamanan,Politik,Politik,Politik,TXT0998
998,Ekonomi,Politik,Politik,Ekonomi,Ekonomi,Ekonomi,Politik,Ekonomi,Ekonomi,Ekonomi,TXT0999


In [ ]:
def majority_vote(row):
    vote_counts = Counter(row)
    most_common = vote_counts.most_common()
    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
        return None  # Voting seimbang
    return most_common[0][0]

# Tandai baris dengan voting seimbang
combined_predictions['final_label'] = combined_predictions.apply(majority_vote, axis=1)

# Tampilkan atau simpan baris-baris dengan voting seimbang
balanced_voting_samples = combined_predictions[combined_predictions['final_label'].isnull()]
balanced_voting_samples.to_csv("balanced_voting_samples.csv", index=False)

In [ ]:
balanced_voting_samples.shape

(25, 12)

In [ ]:
balanced_voting_samples

,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep10lr1e-5bs32,predict using indobert large p2 ep10lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs32,predict using indobert large p2 ep2lr2e-5bs64,predict using indobert large p2 ep2lr3e-5bs32,predict using indobert large p2 ep2lr3e-5bs64,predict using indobert large p2 ep3lr1e-5bs32,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p2 ep3lr2e-5bs64,IDText,final_label
28,Politik,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,Ideologi,Politik,Ideologi,TXT0029,None
47,Ekonomi,Politik,Politik,Ekonomi,Ekonomi,Politik,Politik,Ekonomi,Ekonomi,Politik,TXT0048,None
93,Ideologi,Politik,Politik,Ideologi,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,TXT0094,None
103,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Politik,Politik,TXT0104,None
125,Ideologi,Politik,Politik,Politik,Politik,Politik,Ideologi,Ideologi,Ideologi,Ideologi,TXT0126,None
175,Politik,Politik,Politik,Ekonomi,Ekonomi,Politik,Ekonomi,Ekonomi,Politik,Ekonomi,TXT0176,None
316,Sosial Budaya,Politik,Politik,Ideologi,Sosial Budaya,Ideologi,Ideologi,Ideologi,Sosial Budaya,Sosial Budaya,TXT0317,None
388,Sosial Budaya,Politik,Politik,Politik,Sosial Budaya,Sumber Daya Alam,Sumber Daya Alam,Politik,Sosial Budaya,Sosial Budaya,TXT0389,None
421,Politik,Politik,Politik,Ekonomi,Ekonomi,Politik,Ekonomi,Ekonomi,Politik,Ekonomi,TXT0422,None
432,Sumber Daya Alam,Geografi,Geografi,Sumber Daya Alam,Geografi,Sumber Daya Alam,Geografi,Sumber Daya Alam,Sumber Daya Alam,Geografi,TXT0433,None


In [ ]:
combined_predictions

,labeled_data,predict using indobert large p1 ep3lr1e-5bs32,predict using indobert large p1 ep3lr2e-5bs32,predict using indobert large p1 ep3lr2e-5bs64,predict using indobert large p2 10 epoch,predict using indobert large p2 ep10lr1,predict using indobert large p2 ep3dlr2e-5bs32,predict using indobert large p2 ep3lr2e-5bs32,predict using indobert large p1 ep2lr2e-5bs64,predict using indobert large p1 ep2lr3e-5bs64,IDText,final_label
0,Politik,Politik,Politik,Ideologi,Politik,Politik,Politik,Politik,Politik,Politik,TXT0001,Politik
1,Politik,Pertahanan dan Keamanan,Politik,Pertahanan dan Keamanan,Politik,Politik,Pertahanan dan Keamanan,Politik,Pertahanan dan Keamanan,Pertahanan dan Keamanan,TXT0002,None
2,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,Ideologi,TXT0003,Ideologi
3,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,TXT0004,Politik
4,Ideologi,Ekonomi,Politik,Ekonomi,Ideologi,Ideologi,Sosial Budaya,Sosial Budaya,Ekonomi,Ekonomi,TXT0005,Ekonomi
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Ekonomi,TXT0996,Ekonomi
996,Ideologi,Ideologi,Politik,Ideologi,Politik,Politik,Politik,Ideologi,Politik,Ideologi,TXT0997,None
997,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Politik,Pertahanan dan Keamanan,TXT0998,Politik
998,Ekonomi,Ekonomi,Ekonomi,Ekonomi,Politik,Politik,Politik,Politik,Ekonomi,Politik,TXT0999,None


In [ ]:
combined_predictions.to_csv("voting.csv", index=False)

In [ ]:
combined_predictions[ combined_predictions['final_label'].isnull() ].index

Index([  1,  55, 141, 168, 177, 199, 379, 460, 468, 485, 535, 597, 608, 627,
       777, 788, 823, 849, 872, 888, 926, 954, 972, 996, 998],
      dtype='int64')

In [ ]:
combined_predictions.loc[1]

labeled_data                                                      Politik
predict using indobert large p1 ep3lr1e-5bs32     Pertahanan dan Keamanan
predict using indobert large p1 ep3lr2e-5bs32                     Politik
predict using indobert large p1 ep3lr2e-5bs64     Pertahanan dan Keamanan
predict using indobert large p2 10 epoch                          Politik
predict using indobert large p2 ep10lr1                           Politik
predict using indobert large p2 ep3dlr2e-5bs32    Pertahanan dan Keamanan
predict using indobert large p2 ep3lr2e-5bs32                     Politik
predict using indobert large p1 ep2lr2e-5bs64     Pertahanan dan Keamanan
predict using indobert large p1 ep2lr3e-5bs64     Pertahanan dan Keamanan
IDText                                                            TXT0002
final_label                                                          None
Name: 1, dtype: object

Dikarenakan Terdapat voting yang seimbang, maka terdapat data yang tidak dapat ditentukan labelnya. Untuk data tersebut diberikan label oleh versi parameter model yang memiliki akurasi tertinggi yaitu versi model IndoBert Large P2; epoch = 3; learning rate=2e-5; batch size=64

In [ ]:
# Baca file CSV yang akan digunakan untuk melabel baris dengan voting seimbang
backup_label_file = "hasil csv/predict using indobert large p2 ep3lr2e-5bs64.csv"
backup_labels = pd.read_csv(backup_label_file)['label']

# Isi nilai None pada kolom 'final_label' dengan nilai dari backup_labels
combined_predictions['final_label'] = combined_predictions['final_label'].combine_first(backup_labels)

# Simpan hasil ke dalam file CSV baru
combined_predictions.to_csv("final_predictions.csv", index=False)

In [ ]:
prediksi_none = pd.read_csv('prediksi masih ada none.csv')['final_label']
prediksi_terisi_semua = pd.read_csv('final_predictions.csv')['final_label']

In [ ]:
prediksi_none.head()

0     Politik
1         NaN
2    Ideologi
3     Politik
4     Ekonomi
Name: final_label, dtype: object

In [ ]:
prediksi_terisi_semua.head()

0                    Politik
1    Pertahanan dan Keamanan
2                   Ideologi
3                    Politik
4                    Ekonomi
Name: final_label, dtype: object

In [ ]:
comparison_result = prediksi_none.equals(prediksi_terisi_semua)
print("Apakah prediksi_none dan prediksi_terisi_semua memiliki isi yang persis sama?", comparison_result)

Apakah prediksi_none dan prediksi_terisi_semua memiliki isi yang persis sama? False


In [ ]:
mismatched_rows = prediksi_none !=
print("Baris yang tidak sama antara 'prediksi_none' dan 'prediksi_terisi_semua':")
print(mismatched_rows)

In [ ]:
print(classification_report(ytest, ypred, target_names=class_names))